# Import des librairies

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Scraping de récupération des datasets

In [2]:
pl_url = "https://fbref.com/fr/comps/9/Statistiques-Premier-League"

In [3]:
data_pl = requests.get(pl_url)
soup_pl = BeautifulSoup(data_pl.text)

In [4]:
classement_pl = pd.read_html(data_pl.text, match="Classement du championnat")
classement_pl = classement_pl[0]
#classement_pl

In [5]:
classement = soup_pl.select('table.stats_table')[0]

In [6]:
links_to_club = classement.find_all('a')
links_to_club = [link.get('href') for link in links_to_club]
links_to_club = [link for link in links_to_club if '/equipes/' in link]
#links_to_club

In [7]:
clubs_url = [f"https://fbref.com{link}" for link in links_to_club]
#clubs_url

### Fonction pour enlever tous les accents des noms de joueurs

In [8]:
import re


def remove_accents(raw_text):
    raw_text = re.sub(u"[àáâãäå]", 'a', raw_text)
    raw_text = re.sub(u"[èéêë]", 'e', raw_text)
    raw_text = re.sub(u"[ìíîï]", 'i', raw_text)
    raw_text = re.sub(u"[İ]", 'I', raw_text)
    raw_text = re.sub(u"[òóôõö]", 'o', raw_text)
    raw_text = re.sub(u"[ùúûü]", 'u', raw_text)
    raw_text = re.sub(u"[ýÿ]", 'y', raw_text)
    raw_text = re.sub(u"[ß]", 'ss', raw_text)
    raw_text = re.sub(u"[ñ]", 'n', raw_text)
    raw_text = re.sub(u"[ğ]", 'g', raw_text)
    return raw_text


## Boucles de récupération des rapports de scouting de tous les joueurs

In [10]:
all_players_stats = pd.DataFrame()
for club_url in clubs_url:
    data_club = requests.get(club_url)
    soup_club = BeautifulSoup(data_club.text)
    players_list = pd.read_html(data_club.text, match="Statistiques basiques")
    players_list = players_list[0]
    players_list.columns = players_list.columns.droplevel()
    players_list = players_list.dropna()
    players_list = players_list.drop(columns=["Nation", "Âge", "MJ", "Titulaire", "Min",
                                    "90", "Buts", "PD", "B-PénM", "PénM", "PénT", "CJ", "CR", "B+PD", "B+PD-PénM",
                                    "xG", "npxG", "xA", "npxG+xA", "xG", "xA", "xG+xA", "npxG", "npxG+xA", "Matchs"])
    players_list.insert(0, 'Equipe', club_url.split(
        "/")[-1].replace("Statistiques-", "").replace("-", " "))
    for i in range(len(players_list)):
        players_list['Joueur'][i] = remove_accents(players_list['Joueur'][i])

    list_players_club = soup_club.select('table.stats_table')[0]
    
    links_to_players = list_players_club.find_all('a')
    links_to_players = [player.get('href') for player in links_to_players]
    links_to_players = [
        player for player in links_to_players 
        if '/joueurs/' in player and '/summary/' not in player]
    for link in links_to_players:
        if (link.split("/")[-1].replace("-", " ")) not in players_list.values:
            links_to_players.remove(link)
    players_url = [
            f"https://fbref.com{player}" for player in links_to_players]
    
    for player_url in players_url:
        stats_player = requests.get(player_url)
        try:
            stats_player = pd.read_html(
                stats_player.text, match="Rapport de scouting")
            stats_player = stats_player[0]
            stats_player = stats_player.dropna()
            stats_player.reset_index(drop=True, inplace=True)
            stats_player = stats_player.drop([1, 2, 4, 7, 10, 12, 13])
            stats_player.reset_index(drop=True, inplace=True)
            transposed_stats_player = stats_player.set_index(
                'Statistique').transpose()
            transposed_stats_player.insert(0, 'Joueur', player_url.split(
                "/")[-1].replace("Statistiques-", "").replace("-", " "))
            transposed_stats_player = transposed_stats_player.drop(['Centile'])
            all_players_stats = all_players_stats.append(
                transposed_stats_player)
        except:
            print("no match found")
        time.sleep(2)
    time.sleep(5)

no match found


/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

no match found
no match found
no match found
no match found
no match found
no match found
no match found
no match found
no match found
no match found
no match found


/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

no match found


...

no match found


/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

no match found


/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/3441395640.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_players_stats = all_players_stats.append(


no match found
no match found
no match found
no match found
no match found
no match found
no match found
no match found
no match found
no match found


In [11]:
all_players_stats

Statistique,Joueur,Buts (sans les pénaltys),Passes décisives,npxG+xA,Actions menant à un tir,% de passes réussies,Passes vers l'avant,Dribbles réussis,Pressions,Tacles,Interceptions,Balles contrées,Dégagements,Duels aérien gagnés
Par 90,Joao Cancelo,0.07,0.22,0.28,2.90,84.2%,7.70,1.57,9.60,2.24,2.08,1.26,1.59,1.39
Par 90,Rodri,0.17,0.05,0.11,2.30,92.7%,5.68,0.39,15.43,2.20,1.57,1.38,1.52,2.66
Par 90,Bernardo Silva,0.26,0.17,0.34,3.46,88.5%,2.87,1.47,16.23,1.61,1.07,1.09,0.69,0.40
Par 90,Aymeric Laporte,0.10,0.00,0.14,0.87,94.6%,6.02,0.10,4.58,1.12,1.02,0.85,2.51,2.21
Par 90,Ruben Dias,0.06,0.11,0.15,1.11,93.5%,3.94,0.09,5.82,1.28,1.43,1.17,2.74,1.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Par 90,Ozan Kabak,0.00,0.00,0.02,0.98,82.4%,1.76,1.37,9.18,1.27,2.15,1.85,2.83,2.83
Par 90,Kieran Dowell,0.10,0.10,0.32,2.27,68.7%,2.37,0.39,26.64,1.68,0.89,0.99,0.20,1.28
Par 90,Lukas Rupp,0.00,0.00,0.22,3.16,83.2%,3.39,1.52,20.13,1.17,1.99,0.94,0.47,0.70
Par 90,Adam Idah,0.14,0.14,0.37,1.73,67.6%,1.01,1.73,13.39,1.58,0.58,0.58,1.15,2.30


In [13]:
all_players_stats.to_csv("csv_xl/PL_players_stats.csv")

In [14]:
all_players_stats.to_excel("csv_xl/PL_players_stats.xlsx")


## Liste des noms des joueurs à merge avec leur df de statistiques

In [15]:
players_list_to_merge = pd.DataFrame()
for club_url in clubs_url:
    data_club = requests.get(club_url)
    soup_club = BeautifulSoup(data_club.text)
    players_list = pd.read_html(data_club.text, match="Statistiques basiques")
    players_list = players_list[0]
    players_list.columns = players_list.columns.droplevel()
    players_list = players_list.dropna()
    players_list = players_list.drop(columns=["Nation", "Âge", "MJ", "Titulaire", "Min",
                                              "90", "Buts", "PD", "B-PénM", "PénM", "PénT", "CJ", "CR", "B+PD", "B+PD-PénM",
                                              "xG", "npxG", "xA", "npxG+xA", "xG", "xA", "xG+xA", "npxG", "npxG+xA", "Matchs"])
    players_list.insert(0, 'Equipe', club_url.split(
        "/")[-1].replace("Statistiques-", "").replace("-", " "))
    for i in range(len(players_list)):
        players_list['Joueur'][i] = remove_accents(
            players_list['Joueur'][i])
    players_list_to_merge = players_list_to_merge.append(
        players_list)


/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/49337175.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  players_list_to_merge = players_list_to_merge.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/49337175.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  players_list_to_merge = players_list_to_merge.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/49337175.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  players_list_to_merge = players_list_to_merge.append(
/var/folders/y0/7mmq5kp12fq249dxyngjmwv80000gn/T/ipykernel_76900/49337175.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

In [16]:
players_list_to_merge

,Equipe,Joueur,Pos
0,Manchester City,Ederson,GB
1,Manchester City,Joao Cancelo,DF
2,Manchester City,Rodri,MT
3,Manchester City,Bernardo Silva,"MT,AT"
4,Manchester City,Aymeric Laporte,DF
...,...,...,...
23,Norwich City,Christos Tzolis,"AT,MT"
24,Norwich City,Christoph Zimmermann,DF
25,Norwich City,Tony Springett,"AT,MT"
26,Norwich City,Jon Rowe,"AT,MT"


In [17]:
players_list_to_merge['Pos'] = players_list_to_merge['Pos'].replace("MT,AT", "AT,MT")
players_list_to_merge['Pos'] = players_list_to_merge['Pos'].replace("DF,AT", "AT,DF")
players_list_to_merge['Pos'] = players_list_to_merge['Pos'].replace("DF,MT", "MT,DF")

In [18]:
team_players_stats = players_list_to_merge.merge(
    all_players_stats, how='left', on='Joueur')


## Data Cleaning

In [19]:
team_players_stats = team_players_stats.dropna()

In [20]:
team_players_stats.reset_index(drop=True, inplace=True)

In [21]:
test_df = team_players_stats

In [22]:
test_df['% de passes réussies'] = test_df['% de passes réussies'].str.replace(
    '%', '')

In [23]:
test_df

,Equipe,Joueur,Pos,Buts (sans les pénaltys),Passes décisives,npxG+xA,Actions menant à un tir,% de passes réussies,Passes vers l'avant,Dribbles réussis,Pressions,Tacles,Interceptions,Balles contrées,Dégagements,Duels aérien gagnés
0,Manchester City,Joao Cancelo,DF,0.07,0.22,0.28,2.90,84.2,7.70,1.57,9.60,2.24,2.08,1.26,1.59,1.39
1,Manchester City,Rodri,MT,0.17,0.05,0.11,2.30,92.7,5.68,0.39,15.43,2.20,1.57,1.38,1.52,2.66
2,Manchester City,Bernardo Silva,"AT,MT",0.26,0.17,0.34,3.46,88.5,2.87,1.47,16.23,1.61,1.07,1.09,0.69,0.40
3,Manchester City,Aymeric Laporte,DF,0.10,0.00,0.14,0.87,94.6,6.02,0.10,4.58,1.12,1.02,0.85,2.51,2.21
4,Manchester City,Ruben Dias,DF,0.06,0.11,0.15,1.11,93.5,3.94,0.09,5.82,1.28,1.43,1.17,2.74,1.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,Norwich City,Sam Byram,DF,0.00,0.00,0.10,0.96,78.2,2.19,0.18,12.18,2.45,3.33,2.72,4.47,3.07
375,Norwich City,Ozan Kabak,DF,0.00,0.00,0.02,0.98,82.4,1.76,1.37,9.18,1.27,2.15,1.85,2.83,2.83
376,Norwich City,Kieran Dowell,"AT,MT",0.10,0.10,0.32,2.27,68.7,2.37,0.39,26.64,1.68,0.89,0.99,0.20,1.28
377,Norwich City,Lukas Rupp,MT,0.00,0.00,0.22,3.16,83.2,3.39,1.52,20.13,1.17,1.99,0.94,0.47,0.70


In [24]:
test_df[
    ['Buts (sans les pénaltys)', 'Passes décisives', 'npxG+xA', 
     'Actions menant à un tir', "% de passes réussies", "Passes vers l'avant", 
     "Dribbles réussis", "Pressions", "Tacles", "Interceptions", 
     "Balles contrées", "Dégagements", "Duels aérien gagnés"]] = test_df[['Buts (sans les pénaltys)', 'Passes décisives', 'npxG+xA',
     'Actions menant à un tir', "% de passes réussies", "Passes vers l'avant",
     "Dribbles réussis", "Pressions", "Tacles", "Interceptions",
     "Balles contrées", "Dégagements", "Duels aérien gagnés"]].astype(float)

test_df[
    ['Equipe', 'Joueur', 'Pos']] = test_df[
    ['Equipe', 'Joueur', 'Pos']].astype(str)

test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Equipe                    379 non-null    object 
 1   Joueur                    379 non-null    object 
 2   Pos                       379 non-null    object 
 3   Buts (sans les pénaltys)  379 non-null    float64
 4   Passes décisives          379 non-null    float64
 5   npxG+xA                   379 non-null    float64
 6   Actions menant à un tir   379 non-null    float64
 7   % de passes réussies      379 non-null    float64
 8   Passes vers l'avant       379 non-null    float64
 9   Dribbles réussis          379 non-null    float64
 10  Pressions                 379 non-null    float64
 11  Tacles                    379 non-null    float64
 12  Interceptions             379 non-null    float64
 13  Balles contrées           379 non-null    float64
 14  Dégagement

## 1er export

In [25]:
team_players_stats.to_csv("csv_xl/PL_players_stats_team.csv")

In [26]:
team_players_stats.to_excel("csv_xl/PL_players_stats_team.xlsx")

In [27]:
classement_pl = classement_pl.drop(columns=["xGD", "xGD/90", "5 derniers", "Affluence", "Meilleur buteur de l'équipe",
                                            "Gardien de but", "Notes"])

KeyError: "['5 derniers'] not found in axis"

In [ ]:
classement_pl.set_index('Clt')

,Équipe,MJ,V,N,D,BM,BE,DB,Pts,xG,xGA
Clt,,,,,,,,,,,
1,Manchester City,37,28,6,3,96,24,72,90,86.1,26.8
2,Liverpool,37,27,8,2,91,25,66,89,86.1,33.2
3,Chelsea,37,20,11,6,74,32,42,71,64.9,35.0
4,Tottenham,37,21,5,11,64,40,24,68,61.5,39.0
5,Arsenal,37,21,3,13,56,47,9,66,56.9,46.1
6,Manchester Utd,37,16,10,11,57,56,1,58,53.9,54.3
7,West Ham,37,16,8,13,59,48,11,56,49.8,49.4
8,Wolves,37,15,6,16,37,40,-3,51,34.8,57.0
9,Leicester City,37,13,10,14,58,58,0,49,48.5,59.9


In [28]:
classement_pl.to_csv("csv_xl/classement_pl.csv")

## Recuperation du dataset des gardiens

In [29]:
pl_keepers_url = "https://fbref.com/fr/comps/9/keepers/Statistiques-Premier-League"

In [30]:
data_pl_keepers = requests.get(pl_keepers_url)
soup_pl_keepers = BeautifulSoup(data_pl_keepers.text)

In [31]:
keepers_stats = pd.read_html(
    data_pl_keepers.text, match="Gardiens de but de l'équipe")
keepers_stats = keepers_stats[0]
keepers_stats

Unnamed: 0_level_0 Unnamed: 1_level_0 Temps de jeu                        \
               Équipe               # JC           MJ Titulaire   Min    90   
0             Arsenal                  2           38        38  3420  38.0   
1         Aston Villa                  3           38        38  3420  38.0   
2           Brentford                  3           38        38  3420  38.0   
3            Brighton                  2           38        38  3419  38.0   
4             Burnley                  2           38        38  3420  38.0   
5             Chelsea                  2           38        38  3420  38.0   
6      Crystal Palace                  2           38        38  3420  38.0   
7             Everton                  2           38        38  3420  38.0   
8        Leeds United                  2           38        38  3420  38.0   
9      Leicester City                  2           38        38  3420  38.0   
10          Liverpool                  2           38        38  3420  38.0   
11    Manchester City                  2           38        38  3420  38.0   
12     Manchester Utd                  1           38        38  3420  38.0   
13      Newcastle Utd                  3           38        38  3420  38.0   
14       Norwich City                  2           38        38  3420  38.0   
15        Southampton                  3           38        38  3420  38.0   
16          Tottenham                  1           38        38  3420  38.0   
17            Watford                  2           38        38  3420  38.0   
18           West Ham                  2           38        38  3420  38.0   
19             Wolves                  2           38        38  3420  38.0   

   Performance                    ...                       Pénaltys           \
            BE  BE90  TCC Arrêts  ...   V   N   D  CS   CS%     PénT Pc Pa Pm   
0           48  1.26  141     93  ...  22   3  13  13  34.2        6  5  0  1   
1           54  1.42  145     96  ...  13   6  19  11  28.9        6  4  1  1   
2           56  1.47  173    123  ...  13   7  18   9  23.7        5  5  0  0   
3           44  1.16  145    103  ...  12  15  11  11  28.9        4  3  0  1   
4           53  1.39  172    123  ...   7  14  17   9  23.7        5  5  0  0   
5           33  0.87  108     81  ...  21  11   6  16  42.1        6  6  0  0   
6           46  1.21  141    100  ...  11  15  12  12  31.6        4  3  1  0   
7           66  1.74  184    125  ...  11   6  21   8  21.1        2  2  0  0   
8           79  2.08  219    143  ...   9  11  18   5  13.2        5  5  0  0   
9           59  1.55  181    129  ...  14  10  14   7  18.4       10  7  2  1   
10          26  0.68  105     78  ...  28   8   2  21  55.3        0  0  0  0   
11          26  0.68   85     58  ...  29   6   3  21  55.3        1  1  0  0   
12          57  1.50  177    124  ...  16  10  12   8  21.1        6  3  3  0   
13          62  1.63  162    105  ...  13  10  15   8  21.1        6  5  1  0   
14          84  2.21  212    135  ...   5   7  26   6  15.8       13  9  2  2   
15          67  1.76  180    116  ...   9  13  16   8  21.1        2  2  0  0   
16          40  1.05  133     95  ...  22   5  11  16  42.1        2  2  0  0   
17          77  2.03  180    108  ...   6   5  27   4  10.5        7  6  1  0   
18          51  1.34  159    107  ...  16   8  14   8  21.1        6  2  3  1   
19          43  1.13  157    124  ...  15   6  17  11  28.9       10  9  0  1   

            
   Arrêts%  
0      0.0  
1     20.0  
2      0.0  
3      0.0  
4      0.0  
5      0.0  
6     25.0  
7      0.0  
8      0.0  
9     22.2  
10     NaN  
11     0.0  
12    50.0  
13    16.7  
14    18.2  
15     0.0  
16     0.0  
17    14.3  
18    60.0  
19     0.0  

[20 rows x 21 columns]

In [32]:
keepers_stats.columns.droplevel()

Index(['Équipe', '# JC', 'MJ', 'Titulaire', 'Min', '90', 'BE', 'BE90', 'TCC',
       'Arrêts', 'Arrêts%', 'V', 'N', 'D', 'CS', 'CS%', 'PénT', 'Pc', 'Pa',
       'Pm', 'Arrêts%'],
      dtype='object')

In [33]:
keepers_stats.columns = keepers_stats.columns.droplevel()


In [34]:
keepers_stats = keepers_stats.drop(
    columns=['# JC', 'MJ', 'Titulaire', 'Min', '90', 'V', 'N', 'D', 'CS%'])

In [35]:
keepers_stats.to_csv("csv_xl/keepers_stats.csv")